# Cross validation of the ALS method using Spark
We refer to this tutorial: http://spark.apache.org/docs/latest/ml-tuning.html#example-model-selection-via-cross-validation

The tutorial was not clear: we used SKLearn to split the dataset

# Import

In [112]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from pyspark.mllib.recommendation import ALS
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 100


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import random


In [34]:
from pyspark.sql.functions import col

cross validation imports

In [35]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [63]:
class KFoldIndexes:
    ''' Class to get indexes for cross validation
    
    Usage: 
    indexes=KFoldIndexes(4,10)
    indexes.indexes
    '''
    
    def __init__(self,n_splits,rows):
        test_elements=int(rows/n_splits)
        total_elements=list(range(rows))
        remaining_elements=rows%n_splits
        elements_remaining_list=list(range(rows))
        elements_remaining_number=rows
        
        self.indexes=[]
        
        for i in range(n_splits):
            random.shuffle(elements_remaining_list)
            test=elements_remaining_list[:test_elements+int(remaining_elements>i)]
            train=elements_remaining_list[test_elements+int(remaining_elements>i):]
            elements_remaining_list=train.copy()

            for j in range(i):
                train+=self.indexes[j][1]
            
            
#             test=random.sample(elements_remaining_list, test_elements+int(remaining_elements>i))
            
#             elements_remaining_number-=len(test_indexes)
#             test=[elements_remaining_list[x] for x in test_indexes]
#             elements_remaining_list=[x for x in elements_remaining_list if x not in test]
#             train=[x for x in total_elements if x not in test]
#             print(test_indexes,)

            
            
            self.indexes.append((train,test))
        for i in self.indexes:
            i[0].sort()
            i[1].sort()
        

# Dataframe creation

In [42]:
train = pd.read_csv('../data/data_train.csv')


In [43]:
train['UserID'] = train['Id'].apply(lambda x: int(x.split('_')[0][1:]))
train['MovieID'] = train['Id'].apply(lambda x: int(x.split('_')[1][1:]))
train['Rating'] = train['Prediction']
train = train.drop(['Id', 'Prediction'], axis=1)

In [44]:
train.shape[0]

1176952

In [68]:
k_folds=4
k_fold_indexes=KFoldIndexes(k_folds,train.shape[0])

In [74]:
train.loc[[1,2,3]]

,UserID,MovieID,Rating
1,61,1,3
2,67,1,4
3,72,1,3


In [75]:
def get_tests_database(df,k_fold_indexes):
    tests=[]
    for i in k_fold_indexes.indexes:
        tests.append(df.loc[i[1]])
    return tests

In [93]:
def get_sql_from_pd(df_list):
    sql_list=[]
    for i in df_list:
        print(1)
        sql_list.append(sqlContext.createDataFrame(i))
    return sql_list

In [76]:
tests=get_tests_database(train,k_fold_indexes)

In [94]:
tests_sql=get_sql_from_pd(tests)

1
1
1
1


In [108]:
tr=tests_sql[0].rdd.union(tests_sql[1].rdd).union(tests_sql[2].rdd)

In [110]:
ts=tests_sql[3].rdd

In [113]:
model = ALS.train(tr, 2, 10, 0.1)

In [115]:
validation_for_predict_RDD = ts.map(lambda x: (x[0], x[1]))

In [117]:
predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))

In [119]:
rates_and_preds = ts.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)

In [120]:
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

In [121]:
error

0.9971935506177453

In [95]:
tests_sql

[DataFrame[UserID: bigint, MovieID: bigint, Rating: bigint],
 DataFrame[UserID: bigint, MovieID: bigint, Rating: bigint],
 DataFrame[UserID: bigint, MovieID: bigint, Rating: bigint],
 DataFrame[UserID: bigint, MovieID: bigint, Rating: bigint]]

In [8]:
train_sql = sqlContext.createDataFrame(train)
train_rdd = train_sql.rdd
train_rdd.take(1)

[Row(UserID=44, MovieID=1, Rating=4)]

In [43]:
type(train_sql.map(lambda x: (x[0],x[1],float(x[2]))))

pyspark.rdd.PipelinedRDD

In [52]:
from pyspark.sql.types import DoubleType

In [53]:
train_sql2 = train_sql.withColumn("Rating", train_sql["Rating"].cast(DoubleType()))

In [37]:
train_sql2=sqlContext.createDataFrame(train_sql.map(lambda x: (x[0],x[1],float(x[2]))))

In [54]:
train_sql2.show()

+------+-------+------+
|UserID|MovieID|Rating|
+------+-------+------+
|    44|      1|   4.0|
|    61|      1|   3.0|
|    67|      1|   4.0|
|    72|      1|   3.0|
|    86|      1|   5.0|
|    90|      1|   4.0|
|   108|      1|   3.0|
|   114|      1|   3.0|
|   120|      1|   2.0|
|   135|      1|   5.0|
|   152|      1|   4.0|
|   165|      1|   3.0|
|   182|      1|   3.0|
|   310|      1|   3.0|
|   318|      1|   1.0|
|   333|      1|   3.0|
|   355|      1|   2.0|
|   390|      1|   4.0|
|   401|      1|   4.0|
|   410|      1|   2.0|
+------+-------+------+
only showing top 20 rows



In [30]:
train_sql2[2]

Column<b'_3'>

In [89]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])


als=ALS(userCol='UserID',itemCol='MovieID',)
# We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
# This will allow us to jointly choose parameters for all Pipeline stages.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.
paramGrid = ParamGridBuilder() \
    .addGrid(als.maxIter,[10,20])\
    .build()
#     .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
#     .addGrid(lr.regParam, [0.1, 0.01]) \
#     .build()
    
# metrics = RegressionMetrics()

# Squared Error
# print("MSE = %s" % metrics.meanSquaredError)
# print("RMSE = %s" % metrics.rootMeanSquaredError)

crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(metricName="rmse", 
                                                        labelCol="Rating", predictionCol="Rating" ),
                          numFolds=10)  # use 3+ folds in practice

In [90]:
cvModel=crossval.fit(train_sql2)

In [91]:
type(cvModel)

pyspark.ml.tuning.CrossValidatorModel

In [87]:
cvModel.

[]

In [ ]:
cvModel.params

In [ ]:
train_sql[train_sql.index]

In [ ]:
type(train_sql)

In [ ]:
type(train_rdd)

In [ ]:
type(a.indexes[0][0])

In [14]:
train_dan=train_sql[col('MovieID').isin(a.indexes[0][0])]

NameError: name 'a' is not defined

In [ ]:
test_dan=train_sql[col('MovieID').isin(a.indexes[0][1])]
test_for_prediction=test_dan.rdd.map(lambda x: (x[0], x[1]))

In [ ]:
type(train_dan)

In [ ]:
model = ALS.fit(train_dan, 6, 20, 0.1)

In [ ]:
predictions=model.predictAll(test_for_prediction)

In [ ]:
predictions.take(10)

In [ ]:
train_dan.take(4)

In [ ]:
train_sql.count()

In [ ]:
train_dan.take(10)

In [ ]:
train_dan.count()+test_dan.count()

In [ ]:
test_dan.count()

In [ ]:
test_for_prediction.count()

In [ ]:
predictions.count()

In [ ]:
rates_and_preds = test_dan.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
print(rates_and_preds)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

In [ ]:
error

In [ ]:
validation=train_sql.map(lambda x: (x[0], x[1]))

In [ ]:
model.predictAll(validation).map(lambda r: ((r[0], r[1]), r[2]))

In [ ]:
train_sql.count()

In [ ]:
train_sql.columns

In [ ]:
train_sql[col('UserID').isin([1])].take(6)

# Cross validation

In [ ]:
def bool_mask_from_array(array,max_len):
    mask=[0]*max_len
    for i in array:
        mask[i]=1
    return mask

In [ ]:
random.sample(range(1, 4), 3)

In [26]:
a=KFoldIndexes(4,10)

[1, 8, 4]
[5, 4, 3]
[0, 2]
[0, 1]


In [27]:
a.indexes

[([0, 2, 3, 5, 6, 7, 9], [1, 8, 4]),
 ([0, 1, 2, 3, 4, 8, 9], [7, 6, 5]),
 ([1, 2, 4, 5, 6, 7, 8, 9], [0, 3]),
 ([0, 1, 3, 4, 5, 6, 7, 8], [2, 9])]

In [ ]:
kf = KFold(n_splits=4,rows=1000)

In [ ]:
ranks = [2, 4, 6, 8, 10, 12]
lambdas = [0.1, 0.5, 1.0, 5.0, 10.0]
numIters = [5, 10, 20]
nbr_models = len(ranks)*len(lambdas)*len(numIters)

bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

In [ ]:
def cross_validation(database,rank,lmbda,numIter):
    results=[]
    for train_index,test_index in kf.indexes:
#         mask_train=bool_mask_from_array(train_index,actions.shape[0])
#         mask_graph_train=gl.SArray(mask_train)
#         train=actions[mask_graph_train]
        train=database[col('UserID').isin(train_index)]
        test=database[col('UserID').isin(test_index)]
        test_for_prediction=test.rdd.map(lambda x: (x[0], x[1]))
#         mask_test=bool_mask_from_array(test_index,actions.shape[0])
#         mask_graph_test=gl.SArray(mask_test)
#         test=actions[mask_graph_test]

        model = ALS.train(train.rdd, rank, numIter, lmbda)
        print(train.rdd.take(5))
        predictions=model.predictAll(test_for_prediction).map(lambda r: ((r[0], r[1]), r[2]))
#         print(predictions.take(5))
        print(test.rdd.take(5))
        rates_and_preds = test.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
        print(rates_and_preds)
        error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
        
        results.append(error)
    print(results)
    return np.array(results).mean()

In [ ]:
cross_validation(train_sql, 2, 0.1,5)

In [ ]:
for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    

In [ ]:
als = ALS()
evaluator = MulticlassClassificationEvaluator() # + other params as in Scala    

pipeline = Pipeline(stages=[als])

paramGrid = ParamGridBuilder()\
    .build()
#     .addGrid(als.regParam, [0.1, 0.01]) \
#     .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
#     .build()

numFolds=4

crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=numFolds)

model = crossval.fit(train_sql)

In [ ]:
training_RDD, validation_RDD, test_RDD = train_rdd.randomSplit([6, 2, 2], seed=0)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [ ]:
def computeRMSE(model, data, prediction):
    predictions = model.predictAll(data).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = prediction.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    return error

In [ ]:
ranks = [2, 4, 6, 8, 10, 12]
lambdas = [0.1, 0.5, 1.0, 5.0, 10.0]
numIters = [5, 10, 20]
nbr_models = len(ranks)*len(lambdas)*len(numIters)

bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

In [ ]:
i = 0
for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    try:
        print(rank,numIter,lmbda)
        model = ALS.train(training_RDD, rank, numIter, lmbda)
        validationRmse = computeRMSE(model, validation_for_predict_RDD, validation_RDD)
        print("Model %i/%i: RMSE (validation) = %f" %(i+1, nbr_models, validationRmse))
        print("  Trained with rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))
        print("")
        if (validationRmse < bestValidationRmse):
            bestModel = model
            bestValidationRmse = validationRmse
            bestRank = rank
            bestLambda = lmbda
            bestNumIter = numIter
    except:
        print("Model %i/%i failed!" %(i+1, nbr_models))
        print("  Parameters: rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))

    i += 1
    break
    
# Evaluate the best model on the training set
print("The best model was trained with rank = %d and lambda = %.1f, " % (bestRank, bestLambda) \
  + "and numIter = %d, and its RMSE on the training set is %f" % (bestNumIter, bestValidationRmse))

# Evaluate the best model on the test set
testRmse = computeRMSE(bestModel, test_for_predict_RDD, test_RDD)
print("RMSE on the test set: %f"%(testRmse))